<a href="https://colab.research.google.com/github/HeroBenHero/HeroColabs/blob/main/1TamilMV_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Pre-Requisites**
!pip install seedrcc
import os
import io
import re
import sys
import requests
from time import sleep
from bs4 import BeautifulSoup
from seedrcc import Login,Seedr
from urllib.parse import urljoin
from google.colab import output,files

output.clear()

Username  = "herotesthero1@gmail.com"
Password  = "t&amp;hFsoVFkjoOyw13"
 
account = Login(Username, Password)
account.authorize()
seedr = Seedr(token=account.token)

In [ ]:
#@markdown **Enter The URL To Scrap Magnetic Links**
Movie = "" #@param {type:"string"}
#Language = "Tam" #@param {type:"string"}
# Send a GET request to the web page

Site = "https://www.1tamilmv.tips/" #@param {type:"string"}

response = requests.get(Site)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all the titles with the 'span' tag and 'style' attribute
titles = soup.find_all('span', {'style': 'color:#000000'})

# Loop through each title and print the text content and the URL
for title in titles:
    if Movie.lower() in title.text.lower() and title.text.strip() != '-' and title.find_next('a')['href'][-1] == '0':
        # print(title.text.strip())
        # print(title.find_next('a')['href'])
        URL = title.find_next('a')['href']
        break
else:
  # Find all the titles with the 'a' tag and a style that is not "color:#000000"
  titles = soup.find_all("a", style=lambda x: x != "color:#000000")
  # Loop through each title and print the text content and the URL
  for title in titles:
      if Movie.lower() in title.text.lower() and re.search(r'\(\d{4}\)', title.text) and title["href"][-1] == '0':
          # print(title.text.strip())
          # print(title["href"])
          URL = title["href"]
          break
  else:
    # Find all the links on the page
    links = soup.find_all("a")

    # Iterate over each link and extract the information
    for link in links:
        # Extract the text before the link
        # Extract the text of the link
        if "index.php?/forums/topic/" and '-0' in link['href']:
          link_text = link.text.strip()
          if link_text and link_text[0]!='[':
            # print(link['href'], link_text)
            pass
          else:
            link_text = link.find_previous(text=True)
            if link_text[0]!='-':
              link_text = link_text.strip()
              if len(link_text)!=0:
                # print(link['href'],link_text)
                pass
            # else:
            #   previous_link_text = link.find_previous(text=True).find_previous(text=True).find_previous(text=True)
            #   previous_link_text = previous_link_text.strip()
            #   print(previous_link_text)
          if Movie in link_text:
            URL = link['href']
            break
    else:
      sys.exit("Movie Not Found")

pattern = r'(\d+(?:\.\d+)?)\s*(GB|MB)'
# Send an HTTP request to the web server
response = requests.get(URL)

# Parse the HTML code of the web page
soup = BeautifulSoup(response.text, 'html.parser')

# extract the information you want from the HTML tags
titles = [tag.text.strip() for tag in soup.find_all('strong') if len(tag.text.strip()) > 30 and 'www' not in tag.text.strip()]

# Find all the <a> elements on the page that have a "magnet" href attribute and extract the links
magnetic_links = [link for link in soup.find_all('a', href=lambda x: x and x.startswith('magnet:'))]

link_dict={}
# Print the text of each magnetic link
for title,link in zip(titles, magnetic_links):
  # torrent = link.find_previous(text=True).strip()
  size=re.search(pattern, title)
  if size == None:
    title = link.find_previous(text=True).strip()
    size=re.search(pattern, title)
  # torrent_link = link.find_previous('a')['href']
  # print(torrent_link)
  if float(size[1])>4 and size[2]=='GB':
    pass
  else:
    # print(size[0],torrent)  
    # print(link['href'])
    link_dict[size[0]] = link['href']

while True:
    print('List Of Available Sizes:')
    for i, (torrent, href) in enumerate(link_dict.items()):
        print(f'{i+1}. {torrent}')
    choice = input('\nSelect The File Size To Download : ')
    if not choice.isdigit() or int(choice) not in range(1, len(link_dict) + 1):
        print('Invalid choice. Please try again.')
    else:
        # Execute the download() function using the chosen link
        link = list(link_dict.values())[int(choice)-1]
        break

MagneticURL = link
print()
table=seedr.listContents()
for folder in table['torrents']:
  seedr.deleteFolder(folder["id"])
for folder in table['folders']:
  seedr.deleteFolder(folder["id"])
if table["torrents"]:
  seedr.deleteFolder(table["folder_id"])

flag=1
if 'magnet:?xt=urn:btih:' in MagneticURL:
  table=seedr.addTorrent(MagneticURL)
  if table['result']==True:
    table=seedr.listContents()
    print("Downloading....")
    while(table["torrents"]!=[]):
      for folder in table["torrents"]:
        print('Progress :',folder['progress'])
      sleep(2)
      if folder['warnings']!='[]':
        print('\n\n',folder['warnings'])
        seedr.deleteTorrent(folder["id"])
        flag=0
      table=seedr.listContents()
    if flag:
      for folder in table['folders']:  
        files=seedr.listContents(folder["id"])['files']
      for file in files:
        if file['play_video']==True:
          dl=seedr.fetchFile(file['folder_file_id'])
          print('\nFile Name :\n',dl["name"])
          download=dl["url"]
          if '\xa0' in download:
            download=download.replace("\xa0", "%C2%A0")
          print('\nDownload Link :\n',download)
  else:
    print(table['result'])
else:
  print("Invalid Magnetic URL")